In [1]:
import os
import glob
import math
import re

#from mpmath import *
#from config import *
#from osgeo import gdal

## Conversion of tile to lat and lon in degrees
## This function takes in x, y, and zoom variables and outputs lat and lon in degrees

###### Link to resource: - https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames#Tile_numbers_to_lon./lat._2

In [2]:
def tiletolatlon(xtile, ytile, zoom):
    """
    conversion of tile values to lon and lat in degrees"""
    n = 2.0 ** zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
    lat_deg = lat_rad * 180.0 / math.pi
    return lon_deg, lat_deg

## Conversion of lat-lon to EPSG 3857
##### Link to resource: - https://gis.stackexchange.com/questions/142866/converting-latitude-longitude-epsg4326-into-epsg3857

In [3]:
def ConvertCoor(lon, lat):
    
    """
    Converts lat lon  in degrees to epsg 3857"""

    lonInEPSG3857 = (lon * 20037508.34 / 180)
    latInEPSG3857 = (math.log(math.tan((90 + lat) * math.pi / 360)) / (math.pi / 180)) * (20037508.34 / 180)

    #print("{0},{1}".format(lonInEPSG3857,latInEPSG3857))
    return lonInEPSG3857, latInEPSG3857

## Clipping of tiles to GeoJSON file with the following inputs: - 

- minimum value of x
- minimum value of y
- maximum value of x
- maximum value of y

### Function sends command to clip to command line

In [27]:
def clipping(xmin, ymin, xmax, ymax, fname, n):
    clip = f"ogr2ogr -clipsrc {xmin} {ymin} {xmax} {ymax} -f GeoJSON {fname}.geojson {n}/labels_rep.geojson"
    #os.system(clip)
    #! clip
    print(clip)
    return clip

# Rasterizing of georeferenced tiles file with the following inputs: - 

- minimum value of x
- minimum value of y
- maximum value of x
- maximum value of y
- filename

### Function sends command to clip to command line to rasterize image with provided parameters

In [32]:
def raster(xmin, ymin, xmax, ymax, fname):
    raster = f"gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te {xmin} {ymin} {xmax} {ymax} {fname}.geojson\
    {fname}.tiff"
    os.system(raster)
    print(raster)
    return raster

## Reproject GeoJSON files from EPSG:4326 to EPSG:3857

In [33]:
def label_converter(n):
    conv = f"ogr2ogr -s_srs EPSG:4326 -t_srs EPSG:3857 -f GeoJSON {n}/labels_rep.geojson {n}/labels.geojson"
    os.system(conv)
    #print(conv)
    return conv

## Conversion of rasterized tiff file to png with the following inputs: - 

- filename

### Function sends command to convert using gdal_translate to command line

In [34]:
def tifftopng(fname):
    conv = f"gdal_translate -ot Byte -of PNG -scale -co worldfile=yes {fname}.tiff {fname}-burned.png"
    print(conv)
    return conv

## Below is to run the process of georeferencing and rasterization for every individual tile image in every of the five available folders iteratively with the following steps: -

- iterate through individual folders
- iterate through each image in the current folder
   - georefernce images
   - clip images
   - rasterize images
   - convert from tiff to png

In [55]:
for i in range(5):

    path = f"{i+1}/*?.png"
    print('\033[1m' + f"Performing rasterizing on folder {i+1}" + '\033[0m')
    
    #labels converter
    cmd = label_converter(i+1)  #start on compiling commands to georeference and rasterize tiles in entire folder to
    #be executed in batch files

    #iterate through current folder
    for tiles in glob.glob(path):
        

        fname = tiles.replace(".png", "")
        split = re.split('-', tiles)
        x = int(split[1])
        y = int(split[2])
        z = int(split[3].replace('.png', ''))
        print(x, y, z)
        
        name = list(fname)
        name[0] = f"{i+1}\After"
        name = "".join(name)
        print(name)
        
        #get minimum and maximum lat and lon in degrees
        lon_min, lat_min = tiletolatlon(x, y + 1, z)
        lon_max, lat_max = tiletolatlon(x + 1, y, z)
        print(lon_min, lat_min)
        print(lon_max, lat_max)

        #conversion to espg3857
        x_min, y_min = ConvertCoor(lon_min, lat_min)
        x_max, y_max = ConvertCoor(lon_max, lat_max)
        
        #the following lines can be uncomented to view bounding box
        #print(x_min, y_min)
        #print(x_max, y_max)
        
        
        
        #clipping of tile into geojson
        clip = clipping(x_min, y_min, x_max, y_max, name, i+1)
        cmd += ("&" + clip)
        
        #generate raster of tile image
        ras = raster(x_min, y_min, x_max, y_max, name)
        cmd += ("&" + ras)
        
        #convert tiff to png
        co = tifftopng(name)
        cmd += ("&" + co)
        
        
        print()
    
    print('\033[1m' + f"Commands for rasterizing on folder {i+1}" + '\033[0m')
    #print(cmd)    #complete command to georeference and rasterize entire folders
    #saves joint commands into batch files that can be executed 
    with open(f"{i+1}commands.bat", "w") as f:
        f.write(cmd)
        
    print()
    print()

Performing rasterizing on folder 1
319305 270982 19
1\After\OAM-319305-270982-19
39.24934387207031 -6.057940994289545
39.250030517578125 -6.057258182797312
ogr2ogr -clipsrc 4369216.97320076 -675626.892918624 4369293.410229035 -675550.4558903509 -f GeoJSON 1\After\OAM-319305-270982-19.geojson 1/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4369216.97320076 -675626.892918624 4369293.410229035 -675550.4558903509 1\After\OAM-319305-270982-19.geojson    1\After\OAM-319305-270982-19.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 1\After\OAM-319305-270982-19.tiff 1\After\OAM-319305-270982-19-burned.png

319305 270983 19
1\After\OAM-319305-270983-19
39.24934387207031 -6.058623804918195
39.250030517578125 -6.057940994289545
ogr2ogr -clipsrc 4369216.97320076 -675703.3299468999 4369293.410229035 -675626.892918624 -f GeoJSON 1\After\OAM-319305-270983-19.geojson 1/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4369216

gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4369446.2842855835 -675856.2040034481 4369522.721313858 -675779.7669751741 1\After\OAM-319308-270985-19.geojson    1\After\OAM-319308-270985-19.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 1\After\OAM-319308-270985-19.tiff 1\After\OAM-319308-270985-19-burned.png

319309 270982 19
1\After\OAM-319309-270982-19
39.25209045410156 -6.057940994289545
39.252777099609375 -6.057258182797312
ogr2ogr -clipsrc 4369522.721313858 -675626.892918624 4369599.158342132 -675550.4558903509 -f GeoJSON 1\After\OAM-319309-270982-19.geojson 1/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4369522.721313858 -675626.892918624 4369599.158342132 -675550.4558903509 1\After\OAM-319309-270982-19.geojson    1\After\OAM-319309-270982-19.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 1\After\OAM-319309-270982-19.tiff 1\After\OAM-319309-270982-19-burned.png

319309 270983 19
1\After\OAM-319309-27

gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4369752.032398681 -675856.2040034481 4369828.469426956 -675779.7669751741 1\After\OAM-319312-270985-19.geojson    1\After\OAM-319312-270985-19.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 1\After\OAM-319312-270985-19.tiff 1\After\OAM-319312-270985-19-burned.png

Commands for rasterizing on folder 1
ogr2ogr -s_srs EPSG:4326 -t_srs EPSG:3857 -f GeoJSON 1/labels_rep.geojson 1/labels.geojson&ogr2ogr -clipsrc 4369216.97320076 -675626.892918624 4369293.410229035 -675550.4558903509 -f GeoJSON 1\After\OAM-319305-270982-19.geojson 1/labels_rep.geojson&gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4369216.97320076 -675626.892918624 4369293.410229035 -675550.4558903509 1\After\OAM-319305-270982-19.geojson    1\After\OAM-319305-270982-19.tiff&gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 1\After\OAM-319305-270982-19.tiff 1\After\OAM-319305-270982-19-burned.png&ogr2ogr -clipsrc 4369216.97320076 -

gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 3628159.984079132 39059.32144828994 3628236.4211074067 39135.75847656397 2\After\OAM-309610-261632-19.geojson    2\After\OAM-309610-261632-19.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 2\After\OAM-309610-261632-19.tiff 2\After\OAM-309610-261632-19-burned.png

309610 261633 19
2\After\OAM-309610-261633-19
32.592315673828125 0.35018702872728885
32.59300231933594 0.3508736613849764
ogr2ogr -clipsrc 3628159.984079132 38982.88442001366 3628236.4211074067 39059.32144828994 -f GeoJSON 2\After\OAM-309610-261633-19.geojson 2/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 3628159.984079132 38982.88442001366 3628236.4211074067 39059.32144828994 2\After\OAM-309610-261633-19.geojson    2\After\OAM-309610-261633-19.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 2\After\OAM-309610-261633-19.tiff 2\After\OAM-309610-261633-19-burned.png

309610 261634 19
2\After\OAM-309610-261


386687 220236 19
3\After\OAM-386687-220236-19
85.51689147949219 27.636698718024377
85.517578125 27.63730702015522
ogr2ogr -clipsrc 9519696.812395554 3203246.543900985 9519773.249423828 3203322.980929259 -f GeoJSON 3\After\OAM-386687-220236-19.geojson 3/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 9519696.812395554 3203246.543900985 9519773.249423828 3203322.980929259 3\After\OAM-386687-220236-19.geojson    3\After\OAM-386687-220236-19.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 3\After\OAM-386687-220236-19.tiff 3\After\OAM-386687-220236-19-burned.png

386687 220237 19
3\After\OAM-386687-220237-19
85.51689147949219 27.63609041251195
85.517578125 27.636698718024377
ogr2ogr -clipsrc 9519696.812395554 3203170.106872708 9519773.249423828 3203246.543900985 -f GeoJSON 3\After\OAM-386687-220237-19.geojson 3/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 9519696.812395554 3203170.106872708 9519773.249423828 3

...

gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 9522066.360272065 3202023.5514485906 9522142.797300339 3202099.988476864 3\After\OAM-386718-220252-19.geojson    3\After\OAM-386718-220252-19.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 3\After\OAM-386718-220252-19.tiff 3\After\OAM-386718-220252-19-burned.png

386718 220253 19
3\After\OAM-386718-220253-19
85.53817749023438 27.626357064457633
85.53886413574219 27.626965424068704
ogr2ogr -clipsrc 9522066.360272065 3201947.114420319 9522142.797300339 3202023.5514485906 -f GeoJSON 3\After\OAM-386718-220253-19.geojson 3/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 9522066.360272065 3201947.114420319 9522142.797300339 3202023.5514485906 3\After\OAM-386718-220253-19.geojson    3\After\OAM-386718-220253-19.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 3\After\OAM-386718-220253-19.tiff 3\After\OAM-386718-220253-19-burned.png

386718 220254 19
3\After\OAM-386718-22025

gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 9522295.671356887 3202099.988476864 9522372.108385162 3202176.4255051394 3\After\OAM-386721-220251-19.geojson    3\After\OAM-386721-220251-19.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 3\After\OAM-386721-220251-19.tiff 3\After\OAM-386721-220251-19-burned.png

386721 220252 19
3\After\OAM-386721-220252-19
85.54023742675781 27.626965424068704
85.54092407226562 27.62757378029899
ogr2ogr -clipsrc 9522295.671356887 3202023.5514485906 9522372.108385162 3202099.988476864 -f GeoJSON 3\After\OAM-386721-220252-19.geojson 3/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 9522295.671356887 3202023.5514485906 9522372.108385162 3202099.988476864 3\After\OAM-386721-220252-19.geojson    3\After\OAM-386721-220252-19.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 3\After\OAM-386721-220252-19.tiff 3\After\OAM-386721-220252-19-burned.png

386721 220253 19
3\After\OAM-386721-220253

gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 13832809.006842803 1570551.6199568927 13832885.443871079 1570628.0569851685 4\After\OAM-443114-241596-19.geojson    4\After\OAM-443114-241596-19.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 4\After\OAM-443114-241596-19.tiff 4\After\OAM-443114-241596-19-burned.png

443114 241597 19
4\After\OAM-443114-241597-19
124.26223754882812 13.96738677246825
124.26292419433594 13.968053115152037
ogr2ogr -clipsrc 13832809.006842803 1570475.182928619 13832885.443871079 1570551.6199568927 -f GeoJSON 4\After\OAM-443114-241597-19.geojson 4/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 13832809.006842803 1570475.182928619 13832885.443871079 1570551.6199568927 4\After\OAM-443114-241597-19.geojson    4\After\OAM-443114-241597-19.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 4\After\OAM-443114-241597-19.tiff 4\After\OAM-443114-241597-19-burned.png

443115 241594 19
4\After\OAM-4431

gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4108872.4548976896 -142096.4355623629 4108948.891925964 -142019.99853408878 5\After\OAM-315899-264002-19.geojson    5\After\OAM-315899-264002-19.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 5\After\OAM-315899-264002-19.tiff 5\After\OAM-315899-264002-19-burned.png

315899 264003 19
5\After\OAM-315899-264003-19
36.91062927246094 -1.2770548931316252
36.91131591796875 -1.2763684180848043
ogr2ogr -clipsrc 4108872.4548976896 -142172.8725906364 4108948.891925964 -142096.4355623629 -f GeoJSON 5\After\OAM-315899-264003-19.geojson 5/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4108872.4548976896 -142172.8725906364 4108948.891925964 -142096.4355623629 5\After\OAM-315899-264003-19.geojson    5\After\OAM-315899-264003-19.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 5\After\OAM-315899-264003-19.tiff 5\After\OAM-315899-264003-19-burned.png

315900 264002 19
5\After\OAM-315

gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4109407.514095612 -142096.4355623629 4109483.951123886 -142019.99853408878 5\After\OAM-315906-264002-19.geojson    5\After\OAM-315906-264002-19.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 5\After\OAM-315906-264002-19.tiff 5\After\OAM-315906-264002-19-burned.png

315906 264003 19
5\After\OAM-315906-264003-19
36.915435791015625 -1.2770548931316252
36.91612243652344 -1.2763684180848043
ogr2ogr -clipsrc 4109407.514095612 -142172.8725906364 4109483.951123886 -142096.4355623629 -f GeoJSON 5\After\OAM-315906-264003-19.geojson 5/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4109407.514095612 -142172.8725906364 4109483.951123886 -142096.4355623629 5\After\OAM-315906-264003-19.geojson    5\After\OAM-315906-264003-19.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 5\After\OAM-315906-264003-19.tiff 5\After\OAM-315906-264003-19-burned.png

315907 264002 19
5\After\OAM-31590

In [50]:
print(len(cmd))

9132
